In [23]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
from selenium.webdriver.common.by import By
import os

# 1. 웹페이지 주소와 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36')
driver = webdriver.Chrome(options=options)
URL = 'https://brunch.co.kr'
driver.get(url=URL)
keyword = 'episode_3'
search_keyword = '모더니즘 모네'
prohibited_keyword = '담대한 구상'

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

data_path = f'/Users/yongchanchun/Desktop/MacBook_Pro_Desktop/해커톤/{keyword}'
createFolder(data_path)
driver.implicitly_wait(time_to_wait=1)

# 2. 화면에 나타나는 카테고리 박스를 css를 기준으로 찾아 카테고리 박스의 session 들을 category_boxes 변수에 저장함
search_box = driver.find_element(By.XPATH, "/html/body/div[3]/header/div[1]/div/div[2]/div/button")
search_box.click()
search_box = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div/div[1]/div/div/div/span/input[1]")
search_box.send_keys(search_keyword)
search_box.send_keys(Keys.RETURN)
driver.switch_to.window(driver.window_handles[-1])

# 3. threshold를 기준으로 아래로 스크롤 함
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")
cnt = 0
threshold = 10
while cnt < threshold:
    cnt += 1
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    print(f'*******count:{cnt}*******')

# 7. 화면에 나타나는 게시글을 class를 기준으로 찾아 게시글의 session들을 text_links에 저장함
text_links = driver.find_elements(By.CLASS_NAME, "tit_subject")

count = 1
for text_link in text_links:
    try:
        if prohibited_keyword in text_link.text: 
            continue
        
        # 8. 게시글을 클린한 후 나타나는 새로운 브라우저 탭에 들어감
        text_link.click()
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(0.5)

        # 9. 게시글의 내용을 XPATH를 기준으로 찾은 후 찾은 게시글을 paragraph_list 변수에 저장함 
        paragraph_list = driver.find_elements(By.XPATH, "/html/body/div[3]/div[1]/div[2]/div[1]")

        # 10. 게시글을 corpus 변수에 모두 append 한 후 corpus를 지정된 폴더 경로에 저장함
        corpus = []
        for paragraph in paragraph_list:
            corpus.append(paragraph.text)

        with open(os.path.join(data_path, str(count)+ ".txt"), 'w') as f:
            f.writelines(corpus)

        print(f'successfully created text {count}/{len(text_links)}')
        count = count + 1
        
        while len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[-1])

        # driver.close()
        
    except:
        continue


*******count:1*******
successfully created text 1/6
successfully created text 2/6
successfully created text 3/6
successfully created text 4/6
successfully created text 5/6
